In [1]:
import os
os.chdir('../')
os.getcwd()

'e:\\github_clone\\unsupervised-pre-training-vgg'

In [2]:
from tensorflow_src import vgg_encoder
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import itertools
import tensorflow as tf
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from keras.callbacks import *
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow_src.pipeline import load_data, PlotImg, tf_dataset, process_image
from tensorflow_src.evaluation import batch_prediction

In [3]:
model_path = "models\\vgg_for_brain_cancer.keras"
csv_path = "models\\brain_tumor_2.csv"

In [ ]:
path = ""
train, valid, test = load_data(path)

In [ ]:
print(f"Train:{len(train)} - Valid: {len(valid)} - Test:{len(test)}")

In [ ]:
#tensor dataset
train_df = tf_dataset(train)
valid_df = tf_dataset(valid)
test_df = tf_dataset(test)

In [ ]:
for i,j in train_df.take(1):
    print(i.numpy().shape)

In [ ]:
imgs, labels = next(iter(train_df))
PlotImg(imgs)

# Model

In [4]:
vgg = vgg_encoder((256,256,3))

In [5]:
callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    CSVLogger(csv_path),
    ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=6, min_lr=2e-9, verbose=1),
    EarlyStopping(monitor='val_loss', patience=9, verbose=1, restore_best_weights=True)
]

In [7]:
vgg.compile(optimizer=tf.keras.optimizers.Adam(0.0003), loss='catecorical_crossentropy', metrics=['accuracy'])

In [ ]:
r = model.fit(x=train_df, validation_data=valid_df, epochs=epcochs, callbacks=callbacks)

# Evaluation

In [ ]:
batch_prediction(model_path, test_df, test)